In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import sys
sys.path.insert(0, '../../src/')
from data_loader import load_data, load_game_data
pd.set_option('display.max_columns', None)

In [2]:
path_lineups = "/home/matej/Documents/Projects/nba-data-mining/data/processed/lineups-all-seasons.csv"
path_game_data = "/home/matej/Documents/Projects/nba-data-mining/data/processed/game-data-extended.csv"
path_players = "/home/matej/Documents/Projects/nba-data-mining/data/raw/player-data/player_info.csv"

lineups = pd.read_csv(path_lineups)
game_data = pd.read_csv(path_game_data)
players = pd.read_csv(path_players)
players = players[players["Season"] != "2019-20"]

* * *
### Most common starting lineups
Binary feature (1 = team did start with most common lineup, 0 = team did not start with most common lineup)

In [3]:
game_data["season_name"] = game_data["season_name"].str.split("-").str[0].astype(int)

In [4]:
game_data["home_team_lineup"] = game_data[game_data.columns[63:68]].apply(lambda x: ",".join(x.astype(str)), axis=1)
game_data["visitor_team_lineup"] = game_data[game_data.columns[68:73]].apply(lambda x: ",".join(x.astype(str)), axis=1)

game_data["home_team_common_lineup"] = np.nan
game_data["visitor_team_common_lineup"] = np.nan

In [5]:
df_home_lineups = game_data.groupby(["season_name", "home_team_id"])["home_team_lineup"].agg(pd.Series.mode).astype(str)
df_visitor_lineups = game_data.groupby(["season_name", "visitor_team_id"])["visitor_team_lineup"].agg(pd.Series.mode).astype(str)


In [6]:
for i, row in game_data.iterrows():
    game_data.at[i, "home_team_common_lineup"] = np.asarray(df_home_lineups.iloc[(df_home_lineups.index.get_level_values("season_name") == row["season_name"]) & (df_home_lineups.index.get_level_values("home_team_id") == row["home_team_id"])] == row["home_team_lineup"])
    game_data.at[i, "visitor_team_common_lineup"] = np.asarray(df_visitor_lineups.iloc[(df_visitor_lineups.index.get_level_values("season_name") == row["season_name"]) & (df_visitor_lineups.index.get_level_values("visitor_team_id") == row["visitor_team_id"])] == row["visitor_team_lineup"])

In [7]:
game_data.drop(columns=["home_team_lineup", "visitor_team_lineup"], inplace=True)
game_data["home_team_common_lineup"] = game_data["home_team_common_lineup"].astype(int)
game_data["visitor_team_common_lineup"] = game_data["visitor_team_common_lineup"].astype(int)

In [8]:
game_data

,game_id,play_count,season_name,visitor_team_id,visitor_team_city,visitor_team_nickname,visitor_record_wins,visitor_record_losses,visitor_final_score,home_final_score,home_win,home_team_id,home_team_city,home_team_nickname,home_record_wins,home_record_losses,periods,minutes_played,visitor_players_deployed,home_players_deployed,visitor_fg_made,visitor_fg_missed,visitor_3PT_made,visitor_3PT_missed,home_fg_made,home_fg_missed,home_3PT_made,home_3PT_missed,visitor_ft_made,visitor_ft_missed,home_ft_made,home_ft_missed,visitor_rebound,home_rebound,visitor_team_rebound,home_team_rebound,visitor_turnover,home_turnover,visitor_team_turnover,home_team_turnover,visitor_foul,home_foul,visitor_subs,home_subs,visitor_timeout,home_timeout,visitor_jump_balls_won,home_jump_balls_won,tip_off_winner,visitor_ejection,home_ejection,visitor_team_ejection,home_team_ejection,home_scoring_leader,home_scoring_leader_points,visitor_scoring_leader,visitor_scoring_leader_points,home_made_max_shot_distance,visitor_made_max_shot_distance,home_made_min_shot_distance,visitor_made_min_shot_distance,home_made_mean_shot_distance,visitor_made_mean_shot_distance,home_player1_id,home_player2_id,home_player3_id,home_player4_id,home_player5_id,visitor_player1_id,visitor_player2_id,visitor_player3_id,visitor_player4_id,visitor_player5_id,home_team_common_lineup,visitor_team_common_lineup
0,20000001,429,2000,1610612755,Philadelphia,76ers,1,0,101,72,False,1610612752,New York,Knicks,0,1,4,48.0,12,12,38,28,3,5,25,45,3,8,22,8,19,5,37,37,2,3,13,22,1,0,25,30,20,17,3,7,1,2,HOME_PLAYER,0,0,0,0,275,21.0,947,25.0,26.0,26.0,0.0,0.0,10.720000,9.263158,84.0,275.0,369.0,913.0,948.0,238.0,243.0,248.0,689.0,727.0,1,0
1,20000002,510,2000,1610612739,Cleveland,Cavaliers,1,0,86,82,False,1610612751,New Jersey,Nets,0,1,4,48.0,11,10,32,46,2,5,31,54,3,7,20,12,17,9,52,47,6,6,19,12,0,3,28,31,24,21,6,8,1,1,HOME_PLAYER,0,0,0,0,1425,20.0,441,17.0,27.0,27.0,0.0,0.0,8.129032,9.500000,383.0,950.0,1536.0,1915.0,2030.0,221.0,441.0,980.0,1723.0,1889.0,0,1
2,20000003,478,2000,1610612764,Washington,Wizards,0,1,86,97,True,1610612753,Orlando,Magic,1,0,4,48.0,12,10,33,39,4,3,34,45,6,10,16,8,23,10,44,37,5,7,26,15,1,0,28,24,28,18,7,5,0,1,HOME_PLAYER,0,0,0,0,1503,32.0,1732,16.0,25.0,25.0,0.0,0.0,10.794118,7.696970,255.0,353.0,448.0,680.0,1503.0,393.0,436.0,1732.0,1751.0,1918.0,0,0
3,20000004,448,2000,1610612766,Charlotte,Hornets,1,0,106,82,False,1610612737,Atlanta,Hawks,0,1,4,48.0,11,11,35,27,5,4,30,51,6,9,31,9,16,5,44,29,3,7,17,13,0,0,22,32,17,27,5,5,1,1,HOME_PLAYER,0,0,0,0,673,23.0,469,23.0,26.0,31.0,0.0,0.0,9.133333,8.657143,673.0,1728.0,1891.0,2035.0,2069.0,133.0,136.0,469.0,922.0,1884.0,0,1
4,20000005,505,2000,1610612765,Detroit,Pistons,1,0,104,95,False,1610612761,Toronto,Raptors,0,1,4,48.0,12,11,44,49,3,9,35,58,5,12,13,5,20,11,45,48,9,13,12,15,0,0,27,21,30,18,5,7,1,0,VISITOR_PLAYER,0,0,0,0,1713,26.0,711,44.0,26.0,26.0,0.0,0.0,12.400000,9.454545,213.0,349.0,722.0,891.0,1713.0,182.0,376.0,711.0,1088.0,1112.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22960,21801226,474,2018,1610612760,Oklahoma City,Thunder,49,33,127,116,False,1610612749,Milwaukee,Bucks,60,22,4,48.0,11,8,48,51,23,31,43,57,15,31,8,5,15,6,53,53,1,7,12,14,1,1,20,14,23,17,3,6,1,0,VISITOR_PLAYER,0,0,0,0,204025,29.0,203471,32.0,28.0,28.0,1.0,1.0,12.418605,14.000000,101141.0,1626192.0,1628391.0,1628425.0,1629045.0,201566.0,203471.0,203500.0,203924.0,1628390.0,0,0
22961,21801227,443,2018,1610612742,Dallas,Mavericks,33,49,94,105,True,1610612759,San Antonio,Spurs,48,34,4,48.0,10,13,37,54,11,26,41,47,8,16,9,3,15,3,42,53,4,5,7,10,0,1,19,14,23,28,6,6,0,2,HOME_PLAYER,0,0,0,0,200746,34.0,1717,20.0,27.0,28.0,1.0,1.0,12.341463,14.513514,200746.0,201942.0,1627751.0,1627854.0,1628401.0,1717.0,2734.0,

* * *
### Win/Loss ratio - Most common starting lineups

In [41]:
home_common_won = game_data.groupby(["home_team_common_lineup"])["home_win"].sum()
home_common_lost = game_data.groupby(["home_team_common_lineup"])["home_win"].count() - home_common_won
home_common_won

home_team_common_lineup
0    8951
1    4750
Name: home_win, dtype: int64

In [46]:
visitor_common_lost = game_data.groupby(["visitor_team_common_lineup"])["home_win"].sum()
visitor_common_won = game_data.groupby(["visitor_team_common_lineup"])["home_win"].count() - visitor_common_lost

In [40]:
labels = ["Different lineup", "Most common lineup"]
titles = ["Home team won", "Home team lost"]

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]], subplot_titles=titles)
fig.add_trace(go.Pie(labels=labels, values=home_common_won, name="Home team won"),1, 1)
fig.add_trace(go.Pie(labels=labels, values=home_common_lost, name="Home team lost"),1, 2)
fig.update_layout(width=1500, height=500)
fig.show()

In [47]:
labels = ["Different lineup", "Most common lineup"]
titles = ["Visitor team won", "Visitor team lost"]

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]], subplot_titles=titles)
fig.add_trace(go.Pie(labels=labels, values=visitor_common_won, name="Visitor team won"),1, 1)
fig.add_trace(go.Pie(labels=labels, values=visitor_common_lost, name="Visitor team lost"),1, 2)
fig.update_layout(width=1500, height=500)
fig.show()